In [89]:
from syllabify import generate
import json
from collections import defaultdict
from tqdm import tqdm

In [90]:
with open('../data/word2idx.json') as json_file:  
    word2idx = json.load(json_file)

In [91]:
with open('../data/syll2idx.json') as json_file:  
    syll2idx = json.load(json_file)

In [92]:
idx2syll = {val : key for key, val in syll2idx.items()}
idx2word = {val : key for key, val in word2idx.items()}

In [93]:
unk_emb = syll_emb[syll2idx['UNK']]

In [94]:
if 0 in idx2syll:
    with open('../data/syll_emb.json') as json_file:  
        syll_emb = json.load(json_file)
        
    unk_emb = syll_emb[syll2idx['UNK']]
    del syll_emb[syll2idx['UNK']]
    del syll2idx['UNK']
    
    syll2idx = {key : val + 2 for key, val in syll2idx.items()}
    syll2idx["--NULL--"] = 0
    syll2idx["--OOV--"] = 1
    idx2syll = {val : key for key, val in syll2idx.items()}
    
    syll_emb.insert(0, unk_emb)
    syll_emb.insert(0, [0] * len(syll_emb[0]))
    
    with open('../data/syll2idx.json', 'w') as file:
        file.write(json.dumps(syll2idx))
    with open('../data/syll_emb.json', 'w') as file:
        file.write(json.dumps(syll_emb))
    
    print("Offset all indices by 1. ")

Offset all indices by 1. 


In [125]:
import torch
torch.LongTensor(syll_emb).shape

torch.Size([19945, 64])

In [95]:
idx2syll[0]

'--NULL--'

In [96]:
syll_emb[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [97]:
idx2syll[1]

'--OOV--'

In [98]:
syll_emb[1]

[0.0014307164713864614,
 0.0021365594122892146,
 0.0009235047827434017,
 -0.002938784927050622,
 0.001546670698341199,
 0.006123050894207024,
 -0.005731059461546046,
 -0.006642020163744952,
 -0.0021960306657374806,
 0.007348281494424248,
 0.007290943340438444,
 0.002574536935244502,
 0.0051507707383378135,
 0.004025947947273746,
 0.002468660177383236,
 0.0037806556796870176,
 0.00679073152635239,
 0.004562562887983951,
 0.005493858193688101,
 0.00563448103528915,
 0.004057739914956733,
 -0.007639652687144088,
 -0.0013955127342490693,
 -0.0016969815450679632,
 0.0007192027372602619,
 0.0035806104783647087,
 0.0014903020241238147,
 0.0022611052025641315,
 0.00481063274290057,
 -0.0024338811448644648,
 0.003913951506307441,
 0.00016663605379791703,
 0.007715262471747577,
 0.004148492402889904,
 -0.00637269780897016,
 0.006856086736407665,
 -0.00035711320610042237,
 -0.003870998298113191,
 -0.004620353438435783,
 -0.002021003537623512,
 -0.0021310827159989956,
 0.0014301389193645073,
 -0.0

In [99]:
words = list(word2idx.keys())

In [100]:
syll2idx['--OOV--']

1

In [101]:
word2syll_idx = defaultdict(list)
word_idx2syll_idx = defaultdict(list)

word_missed = 0
syllable_count = 0
syllable_missed = 0

for i, word in enumerate(words):
    syllable = generate(word)
    if not syllable:
        word_missed += 1
        continue
    try:
        for syll in syllable:
            syllable_count += 1
            for s in syll:
                if str(s) in syll2idx: 
                    word2syll_idx[word].append(syll2idx[str(s)])
                    word_idx2syll_idx[word2idx[word]].append(syll2idx[str(s)])
                else:
                    # miss a syllable, replace it with '--OOV--'
                    word2syll_idx[word].append(syll2idx['--OOV--'])
                    word_idx2syll_idx[word2idx[word]].append(syll2idx['--OOV--'])
                    syllable_missed += 1
    except:
        # cannot be broken down into syllables, match entire word to '--OOV--'
        word2syll_idx[word].append(syll2idx['--OOV--'])
        word_idx2syll_idx[word2idx[word]].append(syll2idx['--OOV--'])

#### Sanity check

In [102]:
word2idx['type']

534

In [103]:
word2syll_idx['type']

[2200]

In [104]:
word_idx2syll_idx[534]

[2200]

In [110]:
len(words)

88714

In [115]:
# some words cannot be syllabified
assert(len(word2syll_idx) + word_missed == len(words))

In [116]:
# miss rate is pretty high
100 * word_missed / len(words)

40.40624929548888

In [117]:
# missed syllable
syllable_missed

6986

In [121]:
# existing syllables
import itertools
unique_syll = set(itertools.chain.from_iterable(word2syll_idx.values()))
len(unique_syll)

5087

In [2]:
6986 / (6986 + 5087)

0.5786465667191253

In [122]:
# embedding size (19944 + 1) * syll_dim
len(syll_emb)

19945

#### Syllable check

In [105]:
[idx2syll[idx] for idx in word2syll_idx['type']]

['tayp']

In [106]:
[idx2syll[idx] for idx in word2syll_idx['traditionalists']]

['trah', 'dihsh', 'nah', 'lahsts']

In [107]:
[idx2syll[idx] for idx in word2syll_idx['bioengineer']]

['bay', 'ow', 'ehn', 'jhah', 'nihr']

In [108]:
with open('../data/word2syll_idx.json', 'w') as file:
    file.write(json.dumps(word2syll_idx))

In [109]:
with open('../data/word_idx2syll_idx.json', 'w') as file:
    file.write(json.dumps(word_idx2syll_idx))